--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Install relevant packages and modules

In [18]:
!apt-get install -y dcm2niix # '!' runs commands in the console
!apt-get install -y parallel

# dependencies for freesurfer
!apt-get install -y wget
!apt-get install -y grep
!apt-get install -y tcsh
!apt-get install -y bc

!pip install nibabel

import os
import glob
import nibabel as nib

# install freesurfer (might take a few minutes)

# if you are having issues downloading freesurfer, it is likely due to the version selected here
!wget -O freesurfer.tar.gz https://freesurfer.net/pub/dist/freesurfer/7.4.1/freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz
!tar -xzf freesurfer.tar.gz

# set the relevant freesurfer directories
os.environ['FREESURFER_HOME'] = '/content/freesurfer'
os.environ['SUBJECTS_DIR'] = '/content/freesurfer_output'
os.environ['PATH'] += ':/content/freesurfer/bin'

!source /content/freesurfer/SetUpFreeSurfer.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dcm2niix is already the newest version (1.0.20211006-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
parallel is already the newest version (20210822+ds-2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
grep is already the newest version (3.7-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tcsh is already the newest version

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Add the FreeSurfer License path. This is free to get but due to their policy I cannot include it here. It is available at https://surfer.nmr.mgh.harvard.edu/registration.html

In [25]:
from google.colab import files
license = files.upload()

# change this to what you named the license
your_file_name = 'freesurfer_license.txt'

# set this to
os.environ['FS_LICENSE'] += f':/content/{your_file_name}'

Saving freesurfer_license.txt to freesurfer_license (1).txt


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Get anonymized (T1 weighted) dicom files for analysis, and create temporary folders to store these and other files.

In [26]:
# clone the repository to get the raw dicoms
!git clone https://github.com/datalad/example-dicom-structural

# create folders for the NIfTI conversions and corresponding recons
!mkdir /content/nii_files/
!mkdir /content/freesurfer_output/

'''
Note: "T1 weighting" refers to a form of MRI image processing performed by technicians.
Online resources will nearly always provide processed MRI images,
though they may not be T1 weighted
''';

fatal: destination path 'example-dicom-structural' already exists and is not an empty directory.
mkdir: cannot create directory ‘/content/nii_files/’: File exists
mkdir: cannot create directory ‘/content/freesurfer_output/’: File exists


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Convert the DICOMs to NIfTIs

In [27]:
# prepare paths
INPUT_PATH = '/content/example-dicom-structural/dicoms/'
OUTPUT_PATH = '/content/nii_files/'

# grep to ignore some warnings regarding the manufacturer (since we're using sample dicoms)
!dcm2niix -o '{OUTPUT_PATH}' '{INPUT_PATH}' | grep -v "Unknown manufacturer"

Chris Rorden's dcm2niiX version v1.0.20211006  (JP2:OpenJPEG) GCC11.2.0 x86-64 (64-bit Linux)
Found 384 DICOM file(s)
Convert 384 DICOM as /content/nii_files/dicoms_anat-T1w_20130717141500_401b (274x384x384x1)
Conversion required 0.378637 seconds (0.369799 for core code).


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Reconstruct the NIfTI files.

Parallel allows you to recon multiple subjects simultaneously by recruiting n CPU cores (determined by --jobs n)

In [40]:
# prepare paths
INPUT_PATH = '/content/freesurfer_output'
nii_paths = glob.glob('/content/nii_files/*.nii') # list of paths
ALL_NII = ' '.join(nii_paths)  # format that parallel wants

print(ALL_NII)

# use parallel to execute recon-all on each NIfTI file
!parallel --jobs 2 recon-all -i {} -s {/.} -all ::: /content/nii_files/*.nii

/content/nii_files/dicoms_anat-T1w_20130717141500_401a.nii /content/nii_files/dicoms_anat-T1w_20130717141500_401b.nii /content/nii_files/dicoms_anat-T1w_20130717141500_401.nii
ERROR: You are trying to re-run an existing subject with (possibly)
 new input data (-i). If this is truly new input data, you should delete
 the subject folder and re-run, or specify a different subject name.
 If you are just continuing an analysis of an existing subject, then 
 omit all -i flags.
fs-check-version --s dicoms_anat-T1w_20130717141500_401b --o /tmp/tmp.J4ix1V
Fri Oct 25 11:44:10 PM UTC 2024

setenv SUBJECTS_DIR /content/freesurfer_output
cd /content
/content/freesurfer/bin/fs-check-version --s dicoms_anat-T1w_20130717141500_401b --o /tmp/tmp.J4ix1V
-rwxrwxr-x 1 1623927 1623927 18565 Jun 14  2023 /content/freesurfer/bin/fs-check-version

freesurfer-linux-ubuntu22_x86_64-7.4.1-20230614-7eb8460
$Id$
Linux 2fd6eb4ee372 6.1.85+ #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024 x86_64 x86_64 x86_64 GNU

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------